In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 10]

In [6]:
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import minerl
from tqdm import tqdm

from torch import nn, optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utilities import flatten, unflatten, to_batch_shape, to_torch_channels
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import autoencoder
from AdvancedAutoencoder import AdvancedAutoencoder
from networks import WMAutoencoder, WM_VAE, VisionModel

from dataset_preprocessing import MineRlSequenceDataset


In [7]:
BATCH_SIZE = 256
LEARNING_RATE = 0.0001

In [8]:
dataset = MineRlSequenceDataset("data/npy_obtain_diamond", 1)

In [9]:
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

In [11]:
train_features = next(iter(train_dataloader))
train_features[0].shape

/home/marthinwurer/.pyenv/versions/3.9.6/envs/minerl/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py:64: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return default_collate([torch.as_tensor(b) for b in batch])
/home/marthinwurer/.pyenv/versions/3.9.6/envs/minerl/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py:64: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to pro

torch.Size([256, 1, 3, 64, 64])

In [12]:
model = VisionModel(256)

In [14]:
povs = train_features[0]
batch_size, timesteps, C, H, W = povs.shape
input_data = povs.view(timesteps * batch_size, C, H, W)
input_data.shape

torch.Size([256, 3, 64, 64])

In [15]:
with torch.no_grad():
    outs = model(input_data)

In [16]:
loss_func = autoencoder.abs_loss

In [18]:
loss_func(outs[0], input_data, outs[1])

tensor(0.3129)

In [20]:
model.cuda()

VisionModel(
  (encoder): VisionEncoder(
    (encoder): Sequential(
      (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2))
      (1): ELU(alpha=1.0)
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ELU(alpha=1.0)
      (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2))
      (5): ELU(alpha=1.0)
      (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2))
      (7): ELU(alpha=1.0)
      (8): Flatten(start_dim=1, end_dim=-1)
      (9): Linear(in_features=1024, out_features=256, bias=True)
    )
  )
  (decoder): VisionDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=256, out_features=1024, bias=True)
      (1): Unflatten(dim=-1, unflattened_size=(1024, 1, 1))
      (2): ConvTranspose2d(1024, 128, kernel_size=(5, 5), stride=(2, 2))
      (3): ELU(alpha=1.0)
      (4): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2))
      (5): ELU(alpha=1.0)
      (6): ConvTranspose2d(64, 32, kernel_size=(6, 6), stride=(2, 2))
      (7): ELU(al

In [19]:
losses = []

In [ ]:
with tqdm(enumerate(train_dataloader, 0), total=len(dataset)/(BATCH_SIZE), unit="batch") as t:
    for i, data in t:
        # unpack the data and send it to the gpu
        povs, _, _, _, _, _ = data
        batch_size, timesteps, C, H, W = povs.shape
        images = povs.view(timesteps * batch_size, C, H, W).cuda()
        
        
        